In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
 

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForTokenClassification


2024-12-04 10:12:34.824215: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 10:12:34.825930: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 10:12:34.829573: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 10:12:34.838985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733325154.855286 1449922 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733325154.85

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the dataset
data = pd.read_csv("/home/ahmedabdullahi/NLP590/NLPJobsFinder/Data/modelData.csv")
data = data.dropna(subset=["city_ascii", "iso2", "iso3"])  # Remove missing values

# Preprocess the data
data["labels"] = data["city_ascii"]
unique_labels = data["labels"].unique()
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

def encode_texts_and_labels(df):
    texts = df["iso2"] + " " + df["iso3"] + " " + df["city_ascii"]
    tokenized = tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=64,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    label_ids = [label2id[label] for label in df["labels"]]
    return tokenized["input_ids"], tokenized["attention_mask"], tf.convert_to_tensor(label_ids)

# Split the data
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
X_train, attention_train, y_train = encode_texts_and_labels(train_data)
X_val, attention_val, y_val = encode_texts_and_labels(val_data)

# Adjust target labels for sequence length
sequence_length = 64
y_train = np.repeat(y_train[:, np.newaxis], sequence_length, axis=1)
y_val = np.repeat(y_val[:, np.newaxis], sequence_length, axis=1)

2024-12-04 10:12:42.161890: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# Load the model
model = TFBertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Train the model
history = model.fit(
    {"input_ids": X_train, "attention_mask": attention_train},
    y_train,
    validation_data=(
        {"input_ids": X_val, "attention_mask": attention_val},
        y_val,
    ),
    epochs=3,
    batch_size=16
)

Epoch 1/3
18/18 [==============================] - 108s 5s/step - loss: 14.1820 - accuracy: 1.0851e-04 - val_loss: 13.5981 - val_accuracy: 0.0000e+00
Epoch 2/3
18/18 [==============================] - 92s 5s/step - loss: 12.4684 - accuracy: 0.0039 - val_loss: 16.2476 - val_accuracy: 0.0000e+00
Epoch 3/3
18/18 [==============================] - 92s 5s/step - loss: 11.4361 - accuracy: 0.0064 - val_loss: 18.0772 - val_accuracy: 0.0000e+00
